## École Polytechnique de Montréal
## Département Génie Informatique et Génie Logiciel

## INF8460 – Traitement automatique de la langue naturelle - TP2

## Objectifs d'apprentissage: 

•	Explorer les modèles d’espace vectoriel (vector space models) comme représentations distribuées de la sémantique des mots 
•	Implémenter la fréquence de co-occurrence et la PPMI
•	Comprendre différentes mesures de distance entre vecteurs de mots 
•	Explorer l’intérêt de la réduction de dimensionnalité 



## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Luu Thien-Kim: x% (détail)

Nom Étudiant 2: x% (détail)

Nom Étudiant 3: x% (détail)



## Support de google Colab



In [3]:
# !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !wget https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz

In [4]:
# ! tar -xzf aclImdb_v1.tar.gz
# ! tar -xzf MEN.tar.gz
# ! mkdir -p vsm

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mouradyounes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mouradyounes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Librairies externes

In [2]:
from collections import Counter, defaultdict
from itertools import chain
import csv
import itertools
import numpy as np
import os
import pandas as pd

from nltk.corpus import stopwords as all_stopwords
from nltk import word_tokenize

from scipy.stats import spearmanr
from scipy.spatial.distance import euclidean, cosine
from IPython.display import display
from sklearn.decomposition import TruncatedSVD

## Valeurs globales

In [3]:
DIRNAME_ACL =  os.path.join(os.getcwd(), "aclImdb")
DIRNAME_MEN =  os.path.join(os.getcwd(), "MEN")
DIRNAME_VSM =  os.path.join(os.getcwd(), "vsm")

## 1. Prétraitement (20 points)

**a)**	Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données. Commencez par lire ces données, en gardant séparées les données d'entraînement et de test. La fonction doit mettre les mots en minuscules,  supprimer les stopwords (vous devez utiliser ceux de NLTK) et afficher le nombre total de phrases d’entrainement,  le nombre total de phrases d’entrainement positives et négatives et le nombre total de phrases de test avec le nombre total de phrases de test positives et négatives ;

In [4]:
import os
all_stopwords.words("english")
stopwords_english = set(all_stopwords.words("english"))

TRAIN_POS_DIRECTORY = DIRNAME_ACL + '/train/pos'
TRAIN_NEG_DIRECTORY = DIRNAME_ACL + '/train/neg'
TEST_POS_DIRECTORY = DIRNAME_ACL + '/test/pos'
TEST_NEG_DIRECTORY = DIRNAME_ACL + '/test/neg'

#List[List[]] telle que : [train_pos, train_neg] -> trainData[0][0] : "texte"
trainData = [] 
testData = [] 

#List[] telle que : ["texte1", "texte2", ...] -> train_pos[0] : "texte"
train_pos = []
train_neg = []
test_pos = []
test_neg = []

In [5]:
def getInfo() :
    train_pos_files = os.listdir(TRAIN_POS_DIRECTORY)
    train_neg_files = os.listdir(TRAIN_NEG_DIRECTORY)
    test_pos_files = os.listdir(TEST_POS_DIRECTORY)
    test_neg_files = os.listdir(TEST_NEG_DIRECTORY)
    
    files = [train_pos_files, train_neg_files, test_pos_files, test_neg_files]
    
    training_sentences_nb = 0
    training_sentences_pos_nb = 0
    training_sentences_neg_nb = 0

    testing_sentences_nb = 0
    testing_sentences_pos_nb = 0
    testing_sentences_neg_nb = 0
    
    for listOfFiles in files :
        directory = ""
        if listOfFiles == train_pos_files :
            directory = TRAIN_POS_DIRECTORY
        elif listOfFiles == train_neg_files :
            directory = TRAIN_NEG_DIRECTORY
        elif listOfFiles == test_pos_files :
            directory = TEST_POS_DIRECTORY
        elif listOfFiles == test_neg_files :
            directory = TEST_NEG_DIRECTORY
            
        for file in listOfFiles:
            with open(directory + '/' + file, "r") as f:
                data = list(f)
                data = data[0]
                data = data.lower()
                data = [token for token in nltk.word_tokenize(data) if token not in stopwords_english]
                data = ' '.join(data)
                
                if "train" in directory :
                    if "pos" in directory : 
                        train_pos.append(data)
                        training_sentences_pos_nb += len(nltk.sent_tokenize(data))
                    elif "neg" in directory :
                        train_neg.append(data)
                        training_sentences_neg_nb += len(nltk.sent_tokenize(data))
                    
                elif "test" in directory :
                    if "pos" in directory : 
                        test_pos.append(data)
                        testing_sentences_pos_nb += len(nltk.sent_tokenize(data))
                    elif "neg" in directory :
                        test_neg.append(data)
                        testing_sentences_neg_nb += len(nltk.sent_tokenize(data))
            
            
            trainData.append(train_pos)
            trainData.append(train_neg)
            testData.append(test_pos)
            testData.append(test_neg)
            
    training_sentences_nb = training_sentences_pos_nb + training_sentences_neg_nb
    testing_sentences_nb = testing_sentences_pos_nb + testing_sentences_neg_nb
    
    print("nombre total de phrases d’entrainement : ", training_sentences_nb)
    print("nombre total de phrases d’entrainement positives : ", training_sentences_pos_nb)
    print("nombre total de phrases d’entrainement négatives : ", training_sentences_neg_nb)        
    
    print("nombre total de phrases de test : ", testing_sentences_nb)
    print("nombre total de phrases de test positives : ", testing_sentences_pos_nb)
    print("nombre total de phrases de test négatives : ", testing_sentences_neg_nb)     
    
#     return training_sentences_nb, training_sentences_pos_nb, training_sentences_neg_nb, testing_sentences_nb, testing_sentences_pos_nb, testing_sentences_neg_nb

In [10]:
getInfo()

KeyboardInterrupt: 

In [ ]:
sentence  = "Think and wonder, wonder and think. J'ai trouvé l'a."

tokenizer = nltk.RegexpTokenizer(r"\w+")
new_words = tokenizer.tokenize(sentence)

print(new_words)

**a)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence d'au moins 5 et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne) dans le répertoire aclImdb.

In [ ]:
def build_voc(corpus, unk_cutoff=5):
    # mots = ['red', 'blue', 'red', 'green', 'blue', 'blue']
    # c = Counter(mots)
    # print(c.most_common(5))
    voc = []
    newFilePath = DIRNAME_ACL + '/' +  "vocab.txt"
    count = 0
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    file = open(newFilePath, "w")
    with open(newFilePath, "w") as f: 
        for texts in trainData :
            for text in texts :
                tokens = tokenizer.tokenize(text)
                print(tokens)

                for token in tokens :
                    if tokens.count(token) >= 5 :
                        count += 1
                        if token not in voc :
                          voc.append(token)
                          f.write(token + "\n")

                # count = Counter(tokens)
                # v = count.most_common(unk_cutoff)
    #             print(v)
    #             for unigramme in v :
    # #                 print(unigramme)
    #                 if unigramme[0] not in voc :
    #                     voc.append(unigramme[0])
    #                     f.write(unigramme[0] + "\n")
            
#     print(len(voc))
#     print(len(trainData))
#     return voc

In [ ]:
build_voc(trainData)

## 2. Matrices de co-occurence (30 points)

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

**a)** A partir des textes du corpus d’entrainement (neg/pos), vous devez construire une matrice de co-occurrence mot × mot M(w,w) qui contient les 5000 unigrammes les plus fréquents sous forme de **cadre panda**. Le contexte de co-occurrence est une fenêtre de +/-5 mots autour du mot cible. Le poids est la fréquence de co-occurrence simple. Sauvegardez votre matrice dans un fichier tp2_mat5.csv dans le répertoire vsm.

Attention, le mot lui même de doit pas être compté dans sa co-occurence. Exemple : 
Corpus: [ "I go to school every day by bus", "i go to theatre every night by bus"]

Co-occurence("every", fenetre=2) = [ (to, 2), (by, 2), (school, 1), (day, 1), (theatre, 1), (night, 1), (bus, 0), (every, 0), (go, 0). (i,0) ]

In [6]:
import re
corpus = [ "I go to school every day by bus", "i go to theatre every night by bus"]


In [7]:
def find_vocab(X):
    vocab = []
    for tokens in X:
        words = tokens.split(" ")
        for token in words:
            if token not in vocab:
                vocab.append(token)
    return vocab

In [8]:
def search(text,specificWord):
    m = re.search(r'(\w+ ){0,5}'+specificWord+'( \w+){0,5}', text)
    words = m.group(0).split(" ")
    return words

In [65]:
def OccurenceOfProxWords(vector):
    for i, first in enumerate(vector):
        for second in vector:
            if first[0] == second[0] and first != second:
                vector[i] = (first[0], first[1] + second[1])
                vector.remove(second)
       
    for i, element in enumerate(vector):
        counts = Counter(element[1])
        vector[i] = (element[0], dict(counts))
    return vector

In [66]:
def build_matrix(corpus):

    vector = []
    for phrases in corpus:
        words = phrases.split(" ")
        for token in words:
            wordProx = search(phrases, token)
            wordProx.remove(token)
            surrounding_words = (token,wordProx)
            vector.append(surrounding_words)
          
    vector = OccurenceOfProxWords(vector)
    df1 = pd.DataFrame(vector)
    df2 = pd.json_normalize(df1[1])
    
    cols = df2.columns.tolist()
    for i, element in enumerate(vector):
        cols[i] = element[0]
    df2 = df2[cols]
    
    result = pd.concat([df1, df2], axis=1).drop(1, axis=1).fillna(0)
    print(result)

In [67]:
build_matrix(corpus)

['go', 'to', 'school', 'every', 'day']
['I', 'to', 'school', 'every', 'day', 'by', 'i', 'to', 'theatre', 'every', 'night', 'by']
['I', 'go', 'school', 'every', 'day', 'by', 'bus', 'i', 'go', 'theatre', 'every', 'night', 'by', 'bus']
['I', 'go', 'to', 'every', 'day', 'by', 'bus']
['I', 'go', 'to', 'school', 'day', 'by', 'bus', 'i', 'go', 'to', 'theatre', 'night', 'by', 'bus']
['I', 'go', 'to', 'school', 'every', 'by', 'bus']
['go', 'to', 'school', 'every', 'day', 'bus', 'go', 'to', 'theatre', 'every', 'night', 'bus']
['to', 'school', 'every', 'day', 'by', 'to', 'theatre', 'every', 'night', 'by']
['go', 'to', 'theatre', 'every', 'night']
['i', 'go', 'to', 'every', 'night', 'by', 'bus']
['i', 'go', 'to', 'theatre', 'every', 'by', 'bus']
          0    I   go   to  school  every  day   by  bus    i  theatre  night
0         I  0.0  1.0  1.0     1.0    1.0  1.0  0.0  0.0  0.0      0.0    0.0
1        go  1.0  0.0  2.0     1.0    2.0  1.0  2.0  0.0  1.0      1.0    1.0
2        to  1.0  2.0 

**b)** Calculez maintenant une matrice de cooccurrence mais en ajustant les fréquences basées sur la proximité du mot cible par exemple en les multipliant par 1/𝑑 où d est la distance en jetons (mots) de la cible. Sauvegardez votre matrice (toujours sous forme de cadre panda) dans un fichier tp2_mat5_scaled.csv dans le répertoire vsm.

In [12]:
def distance(words, specificWord):
    distance = []
    for word in words:
        if word != specificWord:
            distanceOfEachWord = (word, 1/abs(words.index(specificWord) - words.index(word)))
            distance.append(distanceOfEachWord)
    return distance

In [104]:
def OccurenceAndDistanceOfProxyWord(vector):

    for i, first in enumerate(vector):
        for second in vector:
            if first[0] == second[0] and first != second:
                vector[i] = (first[0], first[1] + second[1])
                vector.remove(second)
    
    mainVector = []
    for element in vector:
        proxWords_dict = {}
        for word in element[1]:
            if word[0] in proxWords_dict:
                proxWords_dict[word[0]] = proxWords_dict[word[0]]+word[1]+1
            else:
                proxWords_dict[word[0]] = word[1]+1
        element = (element[0], proxWords_dict)
        mainVector.append(element)

    return mainVector

In [109]:
def build_distance_matrix(corpus):
    distanceOfWords = []
    for phrases in corpus:
        words = phrases.split(" ")
        for token in words:
            wordProx = search(phrases, token)
            distanceOfToken = (token, distance(wordProx, token))
            distanceOfWords.append(distanceOfToken)

    distanceOfWords = OccurenceAndDistanceOfProxyWord(distanceOfWords)

    df1 = pd.DataFrame(distanceOfWords)
    df2 = pd.json_normalize(df1[1])
    
    cols = df2.columns.tolist()
    for i, element in enumerate(distanceOfWords):
        cols[i] = element[0]
    df2 = df2[cols]
    
    result = pd.concat([df1, df2], axis=1).drop(1, axis=1).fillna(0)
    print(result)

In [110]:
build_distance_matrix(corpus)

          0         I        go        to    school     every       day  \
0         I  0.000000  2.000000  1.500000  1.333333  1.250000  1.200000   
1        go  2.000000  0.000000  4.000000  1.500000  2.666667  1.250000   
2        to  1.500000  4.000000  0.000000  2.000000  3.000000  1.333333   
3    school  1.333333  1.500000  2.000000  0.000000  2.000000  1.500000   
4     every  1.250000  2.666667  3.000000  2.000000  0.000000  2.000000   
5       day  1.200000  1.250000  1.333333  1.500000  2.000000  0.000000   
6        by  0.000000  2.400000  2.500000  1.333333  3.000000  2.000000   
7       bus  0.000000  0.000000  2.400000  1.250000  2.666667  1.500000   
8         i  0.000000  2.000000  1.500000  0.000000  1.250000  0.000000   
9   theatre  0.000000  1.500000  2.000000  0.000000  2.000000  0.000000   
10    night  0.000000  1.250000  1.333333  0.000000  2.000000  0.000000   

          by       bus         i   theatre     night  
0   0.000000  0.000000  0.000000  0.000000  

**c)**	Vous devez créer une fonction `pmi` qui prend le cadre panda de la matrice $M(w,w)$ et un paramètre boolean flag qui est à True lorsque l'on désire calculer PPMI et à False quand on veut calculer PMI. La fonction transforme la matrice en entrée en une matrice $M’(w,w)$ avec les valeurs PMI ou PPMI selon la valeur du paramètre booléen. La fonction retourne le nouveau cadre panda correspondant. 

Pour une matrice  $X_{m \times n}$:


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


In [ ]:
def pmi(df, flag=True):

**d)** Créer les matrice PMIs et PPMIs en vous basant sur les deux matrices que vous avez déjà créée	Sauvegardez vos matrices dans un fichier tp2_mat5<\_scaled>_{pmi|ppmi}.csv toujours dans le répertoire vsm. 

(votre nom de fichier doit contenir "_scaled" s'il est formé à partir Mww_scaled et "pmi" si le flag est false "ppmi" sinon) 

## 3. Test de PPMI (20 points)

Pour le test des matrices de cooccurrences, nous allons comparer deux mesures de distance entre deux vecteurs, la distance euclidienne et la distance cosinus provenant du module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

\\

**a)**	Implémentez la fonction voisins(mot, pd, distance) qui prend un mot en entrée et une métrique de distance et qui retourne les n mots les plus similaires selon la mesure. Pour un mot w, elle ordonne tous les mots du vocabulaire en fonction de leur distance de w en utilisant la métrique de distance distance (par défaut: cosine)sur le vsm pd. Les mesures de distance à tester sont : la distance Euclidienne et la distance cosinus implantées ci-dessus.

In [ ]:
# def voisins(word, df, distfunc, n):
#     position_word = VOCABULAIRE.index(word) #Trouver la position du mot dans le voc 
#     distance = 0
#     words = []
#     similary_words = []
#     for # parcourir la matrice
#         if distfunc == 'euclidean':
#             distance = distance.euclidean(df,word)
#         elif distfunc == 'cosinus':
#             distance = distance.cosine(df,word)
        
#     common_words = Counter(words).most_common(n) # Si c des tuples recup que le mot 
#     for w in common_words:
#         similary_words.append(w[0])
        
#     return similary_words 

**b)** En utilisant le cadre panda associé aux matrices Mww et Mww scaled, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [ ]:
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance euclidienne est : ")
# print(voisins('beautiful', SVM , 'euclidean', 5))
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance cosinus est : ")
# print(voisins('beautiful',SVM , 'cosine', 5))

**c)** En utilisant les cadres panda associés aux matrices PMIs, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

**d)** En utilisant les cadres panda associés aux matrices PPMIs, trouvez les 5 mots les plus similaires au mot
« beautiful » et affichez-les, pour chacune des deux distances. 

**e)** Que constatez-vous entre la différence de performance de la distance euclidienne et la distance cosinus ? Que constatez-vous entre les différents types de matrices de cooccurrence ?

## 4.	Réduction de dimensionnalité (20 points)

**a)** Ecrivez une fonction lsa qui prend en entrée un cadre panda pd (qui contient votre matrice / vsm) et un paramètre K (qui indique le nombre de dimensions finales), et qui applique LSA avec ce paramètre k sur la matrice et retourne le vsm réduit sous forme de cadre panda.

In [ ]:
def lsa(df, k=100):

**b)** Exécutez lsa sur les cadres panda associés à vos matrices Mww et Mww_scaled avec une dimension k=100

**c)** En utilisant les matrices de co-occurrence (de base et scalés) réduites avec LSA, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les.

d) En utilisant les matrices PMIs et PPMIs réduites avec lsa, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les

**e)** En utilisant sklearn.decomposition.TruncatedSVD, créez les matrices réduites à partir des mêmes matrices que celles de la question précédentes (la matrice pmi et la matrice pmi_scaled) Puis tester ces nouvelles matrices LSA pour trouver les 5 mots les plus similaires au mot « beautiful » 

Ici aussi, nous voulons aussi obtenir des matrices de dimension k=100

f) Commentez vos résultats

## 5. Évaluation (10 points)

Il est temps d’évaluer l’intérêt de nos modèles de vecteurs. Nous allons pour cela utiliser un ensemble de données de similarité de mots (relatedness) The MEN Test Collection, qui se trouve dans le répertoire test. L’ensemble de données contient une paire de mots avec un score de similarité attribué par des humains. En d’autres termes, une ligne (un exemple) de l’ensemble de données est de la forme : \<mot_1> \<mot_2> \<score>.

Pour aligner les distances obtenues avec vos métriques, ce score est converti en nombre réel négatif par la fonction read_test_dataset que vous avez dans le squelette du TP.

La métrique d’évaluation est le coefficient de corrélation de Spearman 𝜌 entre les scores humains et vos distances (voir https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient). 

Nous allons maintenant évaluer les différents vsm obtenus sur l'ensemble de données: MEN_dataset.


#### Fonctions pour lire le jeu de données MEN

In [ ]:
def read_test_dataset(
        src_filename,
        delimiter=','):
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[2]
            score = -float(score)
            yield (w1, w2, score)

In [ ]:
# Retourne un itérable sur le jeu de données MEN
def men_dataset():
    src_filename = os.path.join(
        DIRNAME_MEN, 'MEN_dataset_natural_form_full')
    return read_test_dataset(
        src_filename, delimiter=' ')

In [ ]:
def evaluate(ds, df, distfunc=cosine):
    """
    ds : iterator
       retourne des tuples (word1, word2, score).

    df : pd.DataFrame
        le modèle vsm à évaluer

    distfunc : la mesure de distance entre vecteurs
  
    Retour: le coefficient de correlation de Spearman entre les scores de l'ensemble de données de test 
    et celui du modele vsm qui se présente sous la forme d'un cadre Panda pd avec les colonnes
    ['word1', 'word2', 'score', 'distance'].
    """
    data = []
    for w1, w2, score in ds():
        d = {'word1': w1, 'word2': w2,'score': score}
        if w1 not in df.index or w2 not in df.index:
            continue
        else:
            w1 = df.loc[w1]
            w2 = df.loc[w2] 
        d['distance'] = distfunc(w1, w2)
        data.append(d)

    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, b=data['distance'].values)
    return rho, data

**a)**	Testez chacun de vos modèles vsm (Matrice de base, matrice scalée, les PMIs et PPMIs et toutes les matrices LSA (de base, scalée, pmi, ppmi) en appelant la fonction evaluate avec les deux mesure de distance (euclidienne et cosinus) et affichez vos résultats dans une seule table.

**b)**	Commentez vos résultats d'évaluation